In [1]:
import os
os.getcwd()

'/home/yes24/capstone/추천알고리즘/preprocessed'

In [2]:
os.chdir('/home/yes24/models/research/deep_contextual_bandits')

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import pandas as pd
import tensorflow as tf
import time
from absl import app
from absl import flags

In [4]:
from bandits.core.contextual_bandit import run_contextual_bandit
from bandits.algorithms.linear_full_posterior_sampling import LinearFullPosteriorSampling

In [5]:
from bandits.data.data_sampler import sample_census_data
from bandits.data.data_sampler import one_hot,safe_std,remove_underrepresented_classes,classification_to_bandit_problem

In [6]:
def display_results(algos, opt_rewards, opt_actions, h_rewards, t_init, name):
    """Displays summary statistics of the performance of each algorithm."""

    print('---------------------------------------------------')
    print('---------------------------------------------------')
    print('{} bandit completed after {} seconds.'.format(
        name, time.time() - t_init))
    print('---------------------------------------------------')

    performance_pairs = []
    for j, a in enumerate(algos):
        performance_pairs.append((a.name, np.sum(h_rewards[:, j])))
    performance_pairs = sorted(performance_pairs,
                             key=lambda elt: elt[1],
                             reverse=True)
    for i, (name, reward) in enumerate(performance_pairs):
        print('{:3}) {:20}| \t \t total reward = {:10}.'.format(i, name, reward))

    print('---------------------------------------------------')
    print('Optimal total reward = {}.'.format(np.sum(opt_rewards)))
    print('Frequency of optimal actions (action, frequency):')
    print([[elt, list(opt_actions).count(elt)] for elt in set(opt_actions)])
    print('---------------------------------------------------')
    print('---------------------------------------------------')



In [7]:
num_actions = 5

In [8]:
df_100_fin = pd.read_csv('df100_test_0701.csv')

In [9]:
df_100_fin['purchase'].sum()

706.0

In [10]:
df_context = df_100_fin.iloc[:,6:]
labels = df_100_fin.iloc[:,4].astype('int8').values

In [11]:
df_context

,gender_0.0,gender_1.0,g_age_0.0,g_age_1.0,g_age_2.0,g_age_3.0,g_age_4.0,g_age_5.0,g_age_6.0,most_pref_1.0,...,cat34.0,pub0.0,pub1.0,pub2.0,pub3.0,pub4.0,pri0.0,pri1.0,pri2.0,pri3.0
0,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,1,0
1,1,0,0,0,0,0,1,0,0,0,...,1,0,1,0,0,0,0,0,1,0
2,1,0,0,0,0,0,1,0,0,0,...,1,0,0,1,0,0,0,0,1,0
3,1,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,1,0,0
4,1,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4128,0,1,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
4129,0,1,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
4130,0,1,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
4131,0,1,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0


In [12]:
df_context, labels = remove_underrepresented_classes(df_context, labels)
contexts = df_context.values

Unique classes and their ratio of total: {0: 0.0931526736027099, 1: 0.18872489716912655, 2: 0.17735301233970482, 3: 0.432615533510767, 4: 0.10815388337769175}


In [13]:
sampled_vals = classification_to_bandit_problem(contexts, labels, num_actions)

In [14]:
contexts, rewards, (opt_rewards, opt_actions) = sampled_vals

In [15]:
rewards_list = df_100_fin['purchase'].values.tolist()

In [16]:
rewards2 = rewards.copy()

In [17]:
for i in range(len(rewards_list)):
    if rewards_list[i] == 0:
        rewards2[i] = np.where(rewards2[i]==1,0,rewards2[i])

In [18]:
dataset = np.hstack((contexts, rewards))
context_dim = contexts.shape[1]

In [19]:
dataset2 = np.hstack((contexts, rewards2))
context_dim = contexts.shape[1]

In [20]:
mu = np.load('/home/yes24/capstone/소라공유폴더/mean_0701.npy')
cov = np.load('/home/yes24/capstone/소라공유폴더/cov_0701.npy')

In [21]:
hparams = tf.contrib.training.HParams(num_actions=num_actions)
hparams_linear = tf.contrib.training.HParams(num_actions=num_actions,
                                               context_dim=context_dim,
                                               a0=6,
                                               b0=6,
                                               lambda_prior=0.25,
                                               initial_pulls=0,
                                            mu=mu,
                                            cov=cov)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [22]:
algos = [LinearFullPosteriorSampling('LinFullPost', hparams_linear)]

In [23]:
t_init2 = time.time()
results2 = run_contextual_bandit(context_dim, num_actions, dataset2, algos)


In [24]:
_2, h_rewards2,h_vals2 = results2

In [25]:
len(h_vals2)

4133

In [26]:
h_vals2[0][0]

[-10.897289707237338,
 10.568306609291731,
 11.46056392596493,
 7.934488286205957,
 -11.453784921234451]

In [27]:
new_vals = [val[0] for val in h_vals2]

In [28]:
len(new_vals)

4133

In [29]:
val_df = pd.DataFrame()
for i in range(len(new_vals)):
    val_df = pd.concat([val_df,pd.DataFrame(new_vals[i]).T],axis=0)

In [30]:
val_df['val_max'] = val_df[[0,1,2,3,4]].max(axis=1)

In [34]:
val_df_fin = val_df.reset_index().drop(columns='index')

In [112]:
val_df_fin

,0,1,2,3,4,val_max
0,-10.897290,10.568307,11.460564,7.934488,-11.453785,11.460564
1,-11.177284,0.664533,-15.243008,-8.213388,13.394946,13.394946
2,5.186291,4.254928,-19.627558,-3.934000,19.680115,19.680115
3,6.991094,2.944533,4.056551,-10.957553,-5.455815,6.991094
4,0.123438,-1.449245,-19.631774,-3.954024,26.710585,26.710585
...,...,...,...,...,...,...
4128,-0.037540,0.157973,-0.076464,0.051950,0.045581,0.157973
4129,-0.044042,0.140663,-0.161040,-0.193279,0.063017,0.140663
4130,-0.040722,0.114679,-0.081783,0.113127,0.037018,0.114679
4131,0.093741,-0.084701,-0.074464,-0.221203,0.182171,0.182171


In [114]:
val_df_fin.idxmax(axis=1).rename('idx')

0       2
1       4
2       4
3       0
4       4
       ..
4128    1
4129    1
4130    1
4131    4
4132    3
Name: idx, Length: 4133, dtype: int64

In [116]:
val_df_fin = pd.concat([val_df_fin, val_df_fin.idxmax(axis=1).rename('val_max_idx')],axis=1)

In [51]:
df_100_fin

,account_id,product_id,price,purchase,book_cluster,user_cluster,gender_0.0,gender_1.0,g_age_0.0,g_age_1.0,...,cat34.0,pub0.0,pub1.0,pub2.0,pub3.0,pub4.0,pri0.0,pri1.0,pri2.0,pri3.0
0,113512,89940862,0.0,0.0,4.0,3.0,1,0,0,0,...,0,0,0,0,0,1,0,0,1,0
1,113512,84434768,0.0,0.0,2.0,3.0,1,0,0,0,...,1,0,1,0,0,0,0,0,1,0
2,113512,74393338,0.0,0.0,2.0,3.0,1,0,0,0,...,1,0,0,1,0,0,0,0,1,0
3,113512,90108425,0.0,0.0,3.0,3.0,1,0,0,0,...,0,1,0,0,0,0,0,1,0,0
4,113512,85927898,0.0,0.0,3.0,3.0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4128,16790454,90061659,0.0,0.0,3.0,1.0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
4129,16790454,89222585,14400.0,1.0,3.0,1.0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
4130,16790454,89903437,0.0,0.0,3.0,1.0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
4131,16790454,90114596,0.0,0.0,3.0,1.0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0


In [117]:
df_fin_fin = pd.concat([df_100_fin[['account_id','product_id','book_cluster']],val_df_fin],axis=1)

In [126]:
df_fin_fin.iloc[17,:]

account_id      1.159410e+05
product_id      4.433179e+07
book_cluster    0.000000e+00
0               9.966392e+00
1              -3.396033e-01
2              -5.059838e+00
3               1.418003e+00
4               5.138695e+00
val_max         9.966392e+00
val_max_idx     0.000000e+00
Name: 17, dtype: float64

In [135]:
df_fin_fin_fin = pd.DataFrame()
for i in range(len(df_fin_fin)):
    if df_fin_fin['book_cluster'][i] == df_fin_fin['val_max_idx'][i]:
        df_fin_fin_fin = pd.concat([df_fin_fin_fin,df_fin_fin.iloc[i,:]],axis=1)

In [139]:
df_fin_fin_fin = df_fin_fin_fin.T

In [140]:
account_list = df_fin_fin_fin['account_id'].unique()

In [158]:
recc_list = []
for i in range(len(account_list)):
    recc_list.append([df_fin_fin_fin[df_fin_fin_fin['account_id']==account_list[i]].sort_values('val_max',ascending=False)['product_id'].tolist()[0:24]])

In [159]:
recc_list[0][0]

[44331788.0, 34919942.0, 34919948.0, 67292643.0]

In [160]:
recc_list

[[[44331788.0, 34919942.0, 34919948.0, 67292643.0]],
 [[90114596.0, 90114596.0, 85927898.0, 83549800.0]],
 [[85927898.0, 89707566.0]],
 [[86895523.0, 89904068.0, 85927898.0]],
 [[28325977.0, 85927898.0, 89939883.0, 85927898.0]],
 [[37300128.0, 85927898.0, 72158875.0, 2593734.0, 65566236.0]],
 [[90117505.0,
   90117505.0,
   90076563.0,
   78509219.0,
   78509219.0,
   90076563.0,
   90047286.0,
   90118806.0,
   83492117.0,
   85927898.0]],
 [[90114596.0,
   82131803.0,
   16882003.0,
   85927898.0,
   82131803.0,
   90064965.0,
   90039458.0,
   89375963.0,
   81500000.0,
   89484458.0,
   89958123.0]],
 [[90039222.0, 69724044.0]],
 [[90086954.0, 58787793.0, 86008250.0, 24834032.0, 37525716.0, 13434581.0]],
 [[57242725.0,
   3473847.0,
   29379720.0,
   36925841.0,
   15399250.0,
   4827619.0,
   67094938.0,
   89904068.0,
   241455.0,
   89827405.0,
   89746358.0,
   89869475.0,
   59272030.0,
   90000600.0,
   59272030.0,
   5904828.0,
   90062213.0,
   4299517.0,
   89326029.0,
   

In [161]:
[len(recc[0]) for recc in recc_list]

[4,
 4,
 2,
 3,
 4,
 5,
 10,
 11,
 2,
 6,
 24,
 1,
 8,
 8,
 9,
 4,
 24,
 12,
 13,
 14,
 6,
 4,
 23,
 24,
 4,
 4,
 9,
 1,
 1,
 24,
 7,
 24,
 8,
 1,
 24,
 8,
 24,
 24,
 1,
 4,
 24,
 9,
 24,
 10,
 3,
 2,
 7,
 8,
 6,
 24,
 3,
 2,
 8,
 10,
 5,
 6,
 14,
 19,
 7,
 24,
 6,
 1,
 2,
 8,
 24,
 8,
 5,
 10,
 8,
 21,
 11,
 24,
 7,
 8,
 24,
 13,
 9,
 7,
 9,
 5,
 9,
 4,
 22,
 5,
 1,
 9,
 1,
 2,
 24,
 24,
 24,
 2,
 16]

In [163]:
recc_book_list = [y for x in recc_list for y in x]

In [164]:
recc_book_list = [y for x in recc_book_list for y in x]

In [165]:
recc_book_list = list(set(recc_book_list))

In [166]:
recc_book_list.remove(85927898)

In [167]:
recc_book_list


[88389638.0,
 6318087.0,
 69734406.0,
 74393609.0,
 85030922.0,
 35280911.0,
 29653007.0,
 75552785.0,
 2312211.0,
 89913366.0,
 22865949.0,
 89894948.0,
 241709.0,
 352305.0,
 40560690.0,
 17561653.0,
 90087484.0,
 3342397.0,
 90064965.0,
 90075205.0,
 78860364.0,
 89827405.0,
 89927758.0,
 16777296.0,
 74031185.0,
 90075216.0,
 84189271.0,
 28325977.0,
 71850084.0,
 89333861.0,
 88389733.0,
 86241389.0,
 89987184.0,
 89903225.0,
 12648570.0,
 87629946.0,
 75554940.0,
 88373374.0,
 90003585.0,
 44179587.0,
 28113033.0,
 84891785.0,
 78217357.0,
 90075280.0,
 11546769.0,
 75235476.0,
 75200668.0,
 88371357.0,
 60778656.0,
 4522145.0,
 89553061.0,
 89553062.0,
 12687530.0,
 90106027.0,
 30152890.0,
 90075325.0,
 32723134.0,
 83519680.0,
 76470464.0,
 90085586.0,
 37525716.0,
 90075357.0,
 90075364.0,
 89221351.0,
 34914538.0,
 31076595.0,
 78651636.0,
 30748920.0,
 72159493.0,
 61384972.0,
 89643276.0,
 67133713.0,
 90046741.0,
 84936998.0,
 36925738.0,
 83763499.0,
 59023664.0,
 893074

In [175]:
df_100_fin[df_100_fin['product_id'].isin(recc_book_list)]['purchase'].sum()

391.0

In [171]:
df_fin_fin_fin[df_fin_fin_fin['account_id']==account_list[0]].sort_values('val_max',ascending=False)['product_id'].tolist()[0:24]

[44331788.0, 34919942.0, 34919948.0, 67292643.0]

In [172]:
df_fin_fin_fin[df_fin_fin_fin['account_id']==account_list[0]].sort_values('val_max',ascending=False)['product_id']




17    44331788.0
23    34919942.0
22    34919948.0
25    67292643.0
Name: product_id, dtype: float64

In [173]:
df_fin_fin_fin.sort_values(['account_id','val_max'],ascending=False)[0:50]

,account_id,product_id,book_cluster,0,1,2,3,4,val_max,val_max_idx
4120,16790454.0,37300128.0,1.0,-0.086427,0.499775,0.050252,-0.174359,-0.039236,0.499775,1.0
4119,16790454.0,79297023.0,1.0,-0.024507,0.482987,0.001415,-0.151331,-0.142064,0.482987,1.0
4117,16790454.0,37300128.0,1.0,-0.133613,0.455308,-0.102648,-0.241132,0.043122,0.455308,1.0
4124,16790454.0,37300128.0,1.0,-0.041165,0.447951,-0.060858,-0.287709,0.044051,0.447951,1.0
4108,16790454.0,37300128.0,1.0,-0.132805,0.393513,-0.038143,-0.239686,-0.025957,0.393513,1.0
4103,16790454.0,37300128.0,1.0,-0.094839,0.323402,-0.048594,-0.128818,-0.032212,0.323402,1.0
4088,16790454.0,79297023.0,1.0,-0.069809,0.323316,-0.139896,-0.148668,-0.184082,0.323316,1.0
4073,16790454.0,79297023.0,1.0,-0.046853,0.263619,0.118237,-0.419719,-0.022967,0.263619,1.0
4072,16790454.0,59260220.0,2.0,0.112732,-0.282695,0.255865,-0.229528,0.124067,0.255865,2.0
4100,16790454.0,37300128.0,1.0,-0.028108,0.239709,-0.169376,-0.313396,-0.085323,0.239709,1.0


In [96]:
t_init = time.time()
results = run_contextual_bandit(context_dim, num_actions, dataset, algos)


ValueError: too many values to unpack (expected 2)

In [97]:
_, h_rewards,h_vals = results

In [98]:
display_results(algos, opt_rewards, opt_actions, h_rewards, t_init, 'yes24')

---------------------------------------------------
---------------------------------------------------
yes24 bandit completed after 172.4958164691925 seconds.
---------------------------------------------------
  0) LinFullPost         | 	 	 total reward =     3495.0.
---------------------------------------------------
Optimal total reward = 4133.0.
Frequency of optimal actions (action, frequency):
[[0, 385], [1, 780], [2, 733], [3, 1788], [4, 447]]
---------------------------------------------------
---------------------------------------------------


In [94]:
display_results(algos, opt_rewards, opt_actions, h_rewards2, t_init2, 'yes24')

---------------------------------------------------
---------------------------------------------------
yes24 bandit completed after 1718.75066781044 seconds.
---------------------------------------------------
  0) LinFullPost         | 	 	 total reward =      381.0.
---------------------------------------------------
Optimal total reward = 4133.0.
Frequency of optimal actions (action, frequency):
[[0, 385], [1, 780], [2, 733], [3, 1788], [4, 447]]
---------------------------------------------------
---------------------------------------------------


In [31]:
display_results(algos, opt_rewards, opt_actions, h_rewards2, t_init2, 'yes24')

---------------------------------------------------
---------------------------------------------------
yes24 bandit completed after 93.9321277141571 seconds.
---------------------------------------------------
  0) LinFullPost         | 	 	 total reward =       82.0.
---------------------------------------------------
Optimal total reward = 4133.0.
Frequency of optimal actions (action, frequency):
[[0, 385], [1, 780], [2, 733], [3, 1788], [4, 447]]
---------------------------------------------------
---------------------------------------------------


In [31]:
ts_actions = _.T[0]

In [36]:
len(ts_actions)

4133

In [37]:
len(labels)

4133

In [40]:
len(rewards_list)

4133

In [45]:
rewards_list = np.array(rewards_list)

In [50]:
df_eval = pd.concat([pd.DataFrame(ts_actions),pd.DataFrame(labels),pd.DataFrame(rewards_list)],axis=1)

In [52]:
df_eval.columns=['ts_actions','labels','reward']

In [60]:
rewards_fin = 0
for i in range(len(df_eval)):
    if df_eval['reward'][i] == 1:
        if df_eval['ts_actions'][i] == df_eval['labels'][i]:
            rewards_fin +=1

In [61]:
rewards_fin

615

In [68]:
mean1 = algos[0].mu

In [69]:
cov1 = algos[0].cov

In [75]:
mean2 = algos[0].mu

In [76]:
cov2 = algos[0].cov

In [31]:
algos[0].mu

[array([ 2.94238351e-03, -2.94238351e-03,  0.00000000e+00,  1.86575911e-02,
        -2.70227944e-02, -1.15259194e-02,  2.31228639e-02,  5.91907131e-03,
         0.00000000e+00, -2.09351004e-02,  5.09044305e-03, -2.21060517e-02,
        -1.42324452e-02,  5.62984922e-05, -2.10951052e-02,  2.05857716e-03,
         5.08096442e-03, -8.19246790e-03,  1.84492896e-02,  0.00000000e+00,
         1.05237238e-02,  1.97907673e-02,  0.00000000e+00, -2.15151753e-02,
        -1.46805238e-02, -1.08972048e-02, -4.87409864e-03, -1.37862495e-02,
         2.45011192e-02, -6.91920123e-03,  2.83958830e-02, -2.15381536e-03,
         9.58653860e-03, -4.11546726e-03, -1.32562954e-03,  1.32562954e-03,
         1.69998275e-02, -1.69998275e-02, -1.48153127e-03,  1.00725578e-02,
         8.28318908e-02, -1.44173775e-02,  3.27791961e-02,  3.81069300e-02,
         1.08810215e-02,  1.08113050e-02, -9.18157155e-03,  3.27987480e-02,
         1.06950729e-02,  2.76052955e-02,  5.91797466e-02,  6.10902328e-03,
         3.1

In [174]:
recc_book_list

[88389638.0,
 6318087.0,
 69734406.0,
 74393609.0,
 85030922.0,
 35280911.0,
 29653007.0,
 75552785.0,
 2312211.0,
 89913366.0,
 22865949.0,
 89894948.0,
 241709.0,
 352305.0,
 40560690.0,
 17561653.0,
 90087484.0,
 3342397.0,
 90064965.0,
 90075205.0,
 78860364.0,
 89827405.0,
 89927758.0,
 16777296.0,
 74031185.0,
 90075216.0,
 84189271.0,
 28325977.0,
 71850084.0,
 89333861.0,
 88389733.0,
 86241389.0,
 89987184.0,
 89903225.0,
 12648570.0,
 87629946.0,
 75554940.0,
 88373374.0,
 90003585.0,
 44179587.0,
 28113033.0,
 84891785.0,
 78217357.0,
 90075280.0,
 11546769.0,
 75235476.0,
 75200668.0,
 88371357.0,
 60778656.0,
 4522145.0,
 89553061.0,
 89553062.0,
 12687530.0,
 90106027.0,
 30152890.0,
 90075325.0,
 32723134.0,
 83519680.0,
 76470464.0,
 90085586.0,
 37525716.0,
 90075357.0,
 90075364.0,
 89221351.0,
 34914538.0,
 31076595.0,
 78651636.0,
 30748920.0,
 72159493.0,
 61384972.0,
 89643276.0,
 67133713.0,
 90046741.0,
 84936998.0,
 36925738.0,
 83763499.0,
 59023664.0,
 893074

In [176]:
df_100[df_100['product_id'].isin(recc_book_list)]['price'].sum()

5365690.0

In [177]:
df_100[df_100['product_id'].isin(recc_book_list)]['purchase'].sum()

391.0

In [104]:
df_100

,account_id,product_id,price,purchase,book_cluster,user_cluster
0,113512,89940862,0.0,0.0,4.0,3.0
1,113512,84434768,0.0,0.0,2.0,3.0
2,113512,74393338,0.0,0.0,2.0,3.0
3,113512,90108425,0.0,0.0,3.0,3.0
4,113512,85927898,0.0,0.0,3.0,3.0
...,...,...,...,...,...,...
4128,16790454,90061659,0.0,0.0,3.0,1.0
4129,16790454,89222585,14400.0,1.0,3.0,1.0
4130,16790454,89903437,0.0,0.0,3.0,1.0
4131,16790454,90114596,0.0,0.0,3.0,1.0


In [99]:
df_100 = pd.read_csv('/home/yes24/capstone/영송공유폴더/test_100.csv')
df_100

,account_id,product_id,price,purchase,book_cluster,user_cluster
0,113512,89940862,0.0,0.0,4.0,3.0
1,113512,84434768,0.0,0.0,2.0,3.0
2,113512,74393338,0.0,0.0,2.0,3.0
3,113512,90108425,0.0,0.0,3.0,3.0
4,113512,85927898,0.0,0.0,3.0,3.0
...,...,...,...,...,...,...
4128,16790454,90061659,0.0,0.0,3.0,1.0
4129,16790454,89222585,14400.0,1.0,3.0,1.0
4130,16790454,89903437,0.0,0.0,3.0,1.0
4131,16790454,90114596,0.0,0.0,3.0,1.0


In [13]:
user_fin = pd.read_csv('/home/yes24/capstone/원진공유폴더/일요일에고생이많다/user_dummified_1df_0628.csv')

In [14]:
book_fin = pd.read_csv('/home/yes24/capstone/원진공유폴더/일요일에고생이많다/bookfeature_dummified_0628.csv')
book_fin.drop(columns=book_fin.columns[0],inplace=True)

In [15]:
user100_list = df_100['account_id'].unique()

In [16]:
pro100_list = df_100['product_id'].unique()

In [17]:
user_fin[user_fin['account_id'].isin(user100_list)]

,account_id,gender_0.0,gender_1.0,g_age_0.0,g_age_1.0,g_age_2.0,g_age_3.0,g_age_4.0,g_age_5.0,g_age_6.0,...,most_pref_23.0,most_pref_25.0,most_pref_27.0,most_pref_29.0,most_pref_33.0,most_pref_34.0,new_pref_0,new_pref_1,address_is_0,address_is_1
12301,231022.0,1,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,1,0,0,1
13217,2232676.0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,1,0,0,1
13641,3300415.0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,1
28128,9805328.0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,1
49539,2122933.0,1,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1166997,1569435.0,0,1,0,0,0,0,1,0,0,...,0,0,1,0,0,0,1,0,0,1
1167226,2332587.0,0,1,0,0,0,0,1,0,0,...,0,0,0,0,1,0,1,0,0,1
1204153,3056585.0,1,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,1,0,0,1
1209537,1176113.0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,1,1,0,0,1


In [18]:
book_fin[book_fin['product_id'].isin(pro100_list)]

,product_id,cat1.0,cat2.0,cat3.0,cat4.0,cat5.0,cat7.0,cat8.0,cat9.0,cat10.0,...,cat34.0,pub0.0,pub1.0,pub2.0,pub3.0,pub4.0,pri0.0,pri1.0,pri2.0,pri3.0
236,55868083,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
246,55869065,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
340,55936772,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,1,0
348,55944631,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,1,0
374,55976113,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269937,85030922,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
270155,85108883,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
270172,85109178,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
270304,85112006,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1


In [19]:
user_context_add = pd.merge(df_100,user_fin,on='account_id',how='left')
user_context_add

,account_id,product_id,price,purchase,book_cluster,user_cluster,gender_0.0,gender_1.0,g_age_0.0,g_age_1.0,...,most_pref_23.0,most_pref_25.0,most_pref_27.0,most_pref_29.0,most_pref_33.0,most_pref_34.0,new_pref_0,new_pref_1,address_is_0,address_is_1
0,113512,89940862,0.0,0.0,4.0,3.0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,1
1,113512,84434768,0.0,0.0,2.0,3.0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,1
2,113512,74393338,0.0,0.0,2.0,3.0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,1
3,113512,90108425,0.0,0.0,3.0,3.0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,1
4,113512,85927898,0.0,0.0,3.0,3.0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4128,16790454,90061659,0.0,0.0,3.0,1.0,0,1,0,0,...,0,0,0,0,0,0,0,1,1,0
4129,16790454,89222585,14400.0,1.0,3.0,1.0,0,1,0,0,...,0,0,0,0,0,0,0,1,1,0
4130,16790454,89903437,0.0,0.0,3.0,1.0,0,1,0,0,...,0,0,0,0,0,0,0,1,1,0
4131,16790454,90114596,0.0,0.0,3.0,1.0,0,1,0,0,...,0,0,0,0,0,0,0,1,1,0


In [20]:
df_100_fin = pd.merge(user_context_add, book_fin, on='product_id',how='left')
df_100_fin

,account_id,product_id,price,purchase,book_cluster,user_cluster,gender_0.0,gender_1.0,g_age_0.0,g_age_1.0,...,cat34.0,pub0.0,pub1.0,pub2.0,pub3.0,pub4.0,pri0.0,pri1.0,pri2.0,pri3.0
0,113512,89940862,0.0,0.0,4.0,3.0,1,0,0,0,...,0,0,0,0,0,1,0,0,1,0
1,113512,84434768,0.0,0.0,2.0,3.0,1,0,0,0,...,1,0,1,0,0,0,0,0,1,0
2,113512,74393338,0.0,0.0,2.0,3.0,1,0,0,0,...,1,0,0,1,0,0,0,0,1,0
3,113512,90108425,0.0,0.0,3.0,3.0,1,0,0,0,...,0,1,0,0,0,0,0,1,0,0
4,113512,85927898,0.0,0.0,3.0,3.0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4128,16790454,90061659,0.0,0.0,3.0,1.0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
4129,16790454,89222585,14400.0,1.0,3.0,1.0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
4130,16790454,89903437,0.0,0.0,3.0,1.0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
4131,16790454,90114596,0.0,0.0,3.0,1.0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0


In [63]:
df_100_fin.to_csv('df100_test_0701.csv',index=False)